In [1]:
from numpy import pi, linspace              # Numpy helpers
import taichi as ti                         # GPU Accelleration
import taichi.math as tm                    # It's math module
ti.init(arch = ti.gpu, default_fp=ti.f32, fast_math=True)      # Start the engine

## SOLVE THE PENDULUM --------------------------------------------------------------------

# Derivative function
@ti.func
def f(u, a, b):
    return tm.vec2([u.y,-a*u.y -b*ti.sin(u.x)])

# Solve the rk step
@ti.func
def step_u(u, a:float, b:float, h:float=1e-1):
    k1 = f(u,        a,b)
    # k2 = f(u+k1*h/2, a,b)
    # k3 = f(u+k2*h/2, a,b)
    # k4 = f(u+k3*h,   a,b)

    return u + h*(k1)# + 2*k2 + 2*k3 + k4)/6

# Given a starting point in phase space, find where it ends
@ti.func
def evolve(u0, a:float, b:float, h:float, max_iter:int=1000, threshold:float=1e-6):
    u       = u0
    i       = 0
    #run     = True
    for k in range(max_iter):
        u   = step_u(u, a, b, h)    # Step along the path
        if (tm.sin(u.x)*tm.sin(u.x)) + (u.y*u.y) > threshold :
            i = k
            # run = (i<max_iter) and ((tm.sin(u.x)*tm.sin(u.x)) + (u.y*u.y) > threshold )
    return tm.vec3(u.x,u.y,i)


## SETUP THE VISUAL --------------------------------------------------------------------

# The picture
frames  = 300                                   # Number of frames
n       = 500                                   # Number of Pixels
L       = 3*pi                                  # Length to look at
pixels  = ti.Vector.field(3,ti.f32, (2*n,n))    # Stores the color of each pixel
gui     = ti.GUI("A Phase space", res = (n*2,n), fast_gui=True)

# A colormap
@ti.func
def colormap(u,L):
    # map scalar in [0, 1] to RGB (simple blue-to-red gradient)
    val = (tm.clamp(u[0], -2*L, 2*L) + 2*L)/(4*L)
    return tm.vec3([val, 1 - min(u[2],1000)/1000, 1])
    
# Color each pixel
@ti.kernel
def paint(a:float, b:float, L:float, n:int):
    for i,j in pixels:
        u = evolve(tm.vec2(2*i*L/(2*n) - L,L*j/n - L/2), a, b, 1e-1)
        pixels[i,j] = colormap(u,L)

# Now draw the animation
A = linspace(0.001,1,frames)
i = 0

# [paint(a,1,L,n) for a in A]
while True:
    paint(A[int(abs(i - frames + 1))],1,L,n)
    i = (i + 1)%(2*(frames-1))
    gui.set_image(pixels)
    gui.show()

[Taichi] version 1.7.4, llvm 15.0.7, commit b4b956fd, osx, python 3.10.19
[Taichi] Starting on arch=metal


[I 10/27/25 02:07:45.219 123586] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


RuntimeError: Window close button clicked, exiting... (use `while gui.running` to exit gracefully)